In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 

### THIS IS MEANT TO RUN ON NERO - NEEDS TO BE CHANGED IF YOU RUN LOCALLY
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ccorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

/share/sw/open/anaconda/3/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Get ER CSN's With Included Cultures But Not Other Cultures Where We Have Lab Results For At Least One Ordered Culture  
* This will exclude CSNs where all ordered cultures were cancelled, CHECK to see how many this is later

In [ ]:
query="""
DECLARE culture_names ARRAY<STRING>;

SET culture_names = ["URINE CULTURE", "BLOOD CULTURE (2 AEROBIC BOTTLES)", "BLOOD CULTURE (AEROBIC & ANAEROBIC BOTTLES)", "BLOOD CULTURE (AEROBIC & ANAEROBIC BOTTLE)", "ANAEROBIC CULTURE", "RESPIRATORY CULTURE AND GRAM STAIN", "RESPIRATORY CULTURE", "FLUID CULTURE AND GRAM STAIN", "WOUND CULTURE", "STOOL CULTURE", "CSF CULTURE AND GRAM STAIN", "BLOOD CULT - FIRST SET, VIA PHLEBOTOMY", "BLOOD CULT CENTRAL LINE CATHETER BY NURSE", "WOUND CULTURE AND GRAM STAIN", "WOUND CULTURE AND GRAM STAIN, DEEP / SURGICAL / ASP", "FLUID CULTURE / BB GRAM STAIN", "CYSTIC FIBROSIS RESPIRATORY CULTURE"];

WITH er_admits AS (
SELECT anon_id, pat_enc_csn_id_coded, min(effective_time_jittered_utc) as er_admit_time, max(effective_time_jittered_utc) as er_transfer_out_time
FROM `shc_core.adt`
WHERE pat_class_c = "112" AND pat_service = "Emergency"
GROUP BY anon_id, pat_enc_csn_id_coded),

culture_orders as (SELECT DISTINCT ea.anon_id, ea.pat_enc_csn_id_coded, op.description, op.order_type, op.order_status, op.order_proc_id_coded, op.order_time_jittered_utc, lr.anon_id as lr_anon_id,
CASE WHEN op.description IN UNNEST(culture_names) THEN 1 ELSE 0 END is_included_culture,
CASE WHEN op.description NOT IN UNNEST(culture_names) AND op.order_type LIKE "Microbiology%" THEN 1 ELSE 0 END is_excluded_culture
FROM er_admits ea
LEFT JOIN `shc_core.order_proc` op
USING (anon_id)
LEFT JOIN `shc_core.lab_result` lr
ON op.order_proc_id_coded = lr.order_id_coded
WHERE op.order_type LIKE "Microbiology%" -- ER cultures seem to all have microbiology or microbiology culture order_type. 
AND op.order_time_jittered_utc BETWEEN ea.er_admit_time AND ea.er_transfer_out_time),

-- Get order time for first and last culture ordered during er visit. 
first_last_order_times as (
SELECT pat_enc_csn_id_coded, MIN(order_time_jittered_utc) first_order_time, MAX(order_time_jittered_utc) max_order_time
FROM culture_orders 
GROUP BY pat_enc_csn_id_coded),

-- HAS AT LEAST ONE INCLUDE CULTURE AND NO EXCLUDE CULTRES
c_o_include_flag as (
SELECT pat_enc_csn_id_coded, MAX(is_included_culture) include, MAX(is_excluded_culture) exclude_c
FROM culture_orders 
GROUP BY pat_enc_csn_id_coded),

-- AT LEAST ONE ORDERED CULTURE IN CSN APPEARS IN LAB RESULTS - this is where selection bias could creep in
in_lr_flag as (
SELECT pat_enc_csn_id_coded, MAX(CASE WHEN lr_anon_id IS NOT NULL THEN 1 ELSE 0 END) in_lr
FROM culture_orders 
GROUP BY pat_enc_csn_id_coded)

SELECT DISTINCT flt.*
FROM c_o_include_flag co
INNER JOIN in_lr_flag lrf
USING (pat_enc_csn_id_coded)
INNER JOIN first_last_order_times flt
USING (pat_enc_csn_id_coded)
WHERE co.include = 1 AND co.exclude_c = 0 AND lrf.in_lr = 1

"""